Waymo Data Overview:
Dataset comes in form of roughly 1150 driving segments records saved as tensorflow records file format of size ~ 2TB
each driving segment contains ~ 200 annotated frames which represents 20 seconds driving scene
the annotated data includes records from 5 camera and 5 lidars along side some information related to speed and global positioning of the ego vehicle

To Train a detector: due to large size of the data and since frames in a one segment are quite similar, I subsampled the training data by taking random 10 frames out 200 frames in every segment. 

## Approach 1: VoteNet
I used the official implementation of votenet from facebook research as a starting point.

First: I have first implemented a custom input pipeline to optimally preprocess and feed the Waymo point cloud data into the VoteNet network and test it with default implementation

 

First Trial: using votenet with the default configuration
![title](media/screenshot_one.png)

I run the network over 100 times over single frame

the loss function consists of the voteloss + objectness loss + box loss + semantic class loss

<code> loss = vote_loss + 0.5 * objectness_loss + box_loss + 0.1 * sem_cls_loss </code>

This is a screenshot from the prediction over the same frame
![title](media/lidar_prediction_viz.png)
as shown, the network failed to overfit on a single frame, orange and green colors are the ground truth and cyan is the predictions. the prediction boxes are scattered around the scenes in a random places and not even near by the groung truth

I've tried learning rates 1e-3 - 1e-4, I'm using Adam Optimizer, There is a slight difference in curves, thought the loss doesn't go down any further using a wide range of learning rates values [0.01, 0.0001]

no regularizers weight decay = 0
optim.Adam(net.parameters(), lr=BASE_LEARNING_RATE, weight_decay=0)


I also switched off the batchnorm layers which are used in the set abstraction modules in PointNet++ backbone

different plots for the training curves that shows that the network still doesn't fit well to a single frame

VoteNet has only been tuned and tested on RGB-D datasets with indoor 3D scenes: ScanNet and SUN RGB-D, I have also partially reimplemented and adjusted the VoteNet architecture in order to properly account for the characteristics of Waymo outdoor point clouds.

In order to adequately reflect this in the network architecture, I adapt the receptive field radii and increase the number of clusters for feature aggregation. point cloud from Waymo has strongly varying density and is generally more sparse than the point clouds produced from the RGB-D. Since Point features extracted from sparse regions may generalize poorly to dense regions, I used set abstraction modules of the backbone network with multi-scale grouping (MSG) layer in order to capture fine details of point cloud. 

Trial 2: Multi Scale Grouping Backbone

![title](media/lr_0.001x200_lr_0.0001_no_bn.png)
Learning rate 0.01, Adam Optimzier, No weight decay, to remove any sort of regularization, I removed also the Batch norm layers are disabled in set abstraction modules
![title](media/lr_0.0001.png)
Learning rate 0.0001, Adam Optimzier, No weight decay
![title](media/lr_0.0005.png)
Learning rate 0.0005, Adam Optimzier, No weight decay

As shown above, playing with different parameters didn't yield better results to overfit

yet, the loss didn't go near zero !

## Approach 2: Discarding Votenet and use PointNet++ and directly regress the bounding boxes

I Tried PointNet++ and directly regress the bounding box, the below plot shows the result of **overfitting over one frame**

![title](media/pointnet_overfit.png)

in this case the overfitting seems reasonable so I tried to test if this will generalize or not, so I prepared Mini dataset sampled from Waymo, by choosing 100 random frames from 10 random segments and test the generalization over them. I also carefully chosen this dataset by removing empty scenes and train only most common classes (Vehicles, cyclists and Pedestrians) and removing classes stationary classes with low count or significantly different sizes like signs class

![title](media/10_frames_plot.png)
I scaled up the training to 10 frames and validation over one frame, based on the fact the training loss is decreasing in this plot, I increased the no of epochs and used learning rate scheduler to decrease the learning rate when the loss stabilize


![title](media/lr_0.001_65epoch_0.0001.png)


At this point, it fails also to generalize with only pointNet !

Everytime I try to generalize by adding different scenes to the training data, the network predicts so many bounding boxes in the scene like in the below screenshot and fails to acoomodate to any of the training scenes

![title](media/corr_scene.png)

Next step? change the detector?